### - Import modules

In [1]:
import os
import pickle

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns

import re
import datetime
import string
from pykospacing import spacing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Komoran

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
import lightgbm
from lightgbm import LGBMRegressor, Dataset, LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore')

### - Define MAPE formula

In [2]:
def mape(actual, pred): 
    return np.mean(np.abs((actual - pred) / actual)) * 100

### - Load Big Contest data

In [3]:
trainset = pd.read_excel('./data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', skiprows=1)
testset = pd.read_excel('./data/02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', skiprows=1)
watching_rate_data = pd.read_excel('./data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx', skiprows=1)

### - Load external data

In [4]:
weather = pd.read_csv('./data/external_data/weather/weather_df.csv').iloc[:,1:]
munzi = pd.read_csv('./data/external_data/weather/munzi_pm10_utf.csv')
brand_df = pd.read_excel('./data/external_data/product_info/brand.xlsx').iloc[:,1:]
trend_df = pd.read_csv('./data/external_data/product_info/trend검색량.csv')
trend_dict = pd.read_excel('./data/external_data/product_info/trend_dict.xlsx')
search_df = pd.read_csv('./data/external_data/product_info/상품검색량.csv')
search_dict = pd.read_excel('./data/external_data/product_info/search_dict.xlsx')

### - Define functions

#### (1) for basic preprocessing

In [5]:
def preprocessing(df, train=True) :

    # -- Column명 영어로 바꾸기
    df.columns = ['datetime', 'duration', 'mcode', 'pcode', 'pname', 'category', 'price', 'sales']
    
    # -- 노출(분)의 결측치를 앞의 값으로 채우기
    df['duration'].fillna(method='ffill', inplace=True)
    
    if train :
        # -- sales_0 : 원래 취급액이 0이었는지의 여부를 나타내는 칼럼
        df['sales_0'] = [0 if pd.notna(i) else 1 for i in df.sales]
        # 취급액이 0이었으면, 해당 상품 가격으로 치환 / 그렇지 않으면 취급액 그대로 둠
        df['sales'] = [s if s0 == 0 else p for s0, s, p in df[['sales_0', 'sales', 'price']].values]        
        
    return df

#### (2) to split datetime

In [6]:
def date_split(df, train=True) :
    year = []; month = []; day= []; hour = []; minute = []; weekday = [];
    days = ['mon', 'tue', 'wed', 'thurs', 'fri', 'sat', 'sun']
    for i in df.datetime :
        year.append(i.year)
        month.append(i.month)
        day.append(i.day)
        hour.append(i.hour)
        minute.append(i.minute)
        weekday.append(days[i.weekday()])

    for k, v in {'year':year, 'month':month, 'day':day, 'hour':hour, 'minute':minute, 'weekday':weekday}.items() :
        df[k] = pd.Series(v)

    df['hm'] = pd.Series([str(i).rjust(2, '0')+':'+str(j).rjust(2, '0') for i, j in zip(hour, minute)])

    if train :
        return df[['datetime', 'year', 'month', 'day', 'hour', 'minute', 'hm', 'weekday', 
                   'duration', 'mcode', 'pcode', 'pname', 'category', 'price', 'sales', 'sales_0']]
    else :
        return df[['datetime', 'year', 'month', 'day', 'hour', 'minute', 'hm', 'weekday', 
                   'duration', 'mcode', 'pcode', 'pname', 'category', 'price', 'sales']]

#### (3) to make day_order, repeat_order, pgm_order

In [7]:
def day_pgm_repeat_order(df, train=True) :
    if train :
        date_unique = df.datetime.dt.date.unique().tolist()
        trouble_no = [100271, 100270, 100251, 100250, 100272, 100271, 100846, 
                      100845, 100189, 100190, 100271, 100270, 100036, 100038, 
                      100039, 100187, 100189]
        a = [100270, 100271, 100272]; b = [100250, 100251]
        c = [100845, 100846]; d = [100187, 100189, 100190]; e = [100036, 100038, 100039]
        
        df_ = []; day_od = 1
        for i in range(len(date_unique)-1) :
            # print(i)
            date = date_unique[i]
            date_1 = date_unique[i+1]
            if i < 304 :
                m=0; m_1=0;
            elif date == datetime.date(2019, 11, 26) :
                m=0; m_1=20;
                # 2019년 11월 26일은 유일하게 6:00 방송 시작해서, 2:00에 마지막 방송을 함
                # (원래 6:00 방송 시작하면 1:40이 막방)
            else :
                m=20; m_1=20;
            temp = df[df.datetime >= datetime.datetime(
                date.year, date.month, date.day, 6, m)]
            temp = temp[temp.datetime < datetime.datetime(
                date_1.year, date_1.month, date_1.day, 2, m_1)]
        
            # mcode가 같으면 같은 숫자
            cnt=1; mcode_before=0; values_per_day=[]
            for i, value in enumerate(temp.values.tolist(), 1) :
                if mcode_before == 0 :
                    values_per_day.append(value+[cnt, day_od])
                    mcode_before = value[9]
                elif value[9] in trouble_no:
                    one_before = temp.values.tolist()[i-2][9]
                    if (one_before not in trouble_no) \
                        or ((value[9] in a) and (one_before not in a)) \
                            or ((value[9] in b) and (one_before not in b)) \
                                or ((value[9] in c) and (one_before not in c)) \
                                    or ((value[9] in d) and (one_before not in d)) \
                                        or ((value[9] in e) and (one_before not in e)) :
                            cnt += 1
                    values_per_day.append(value+[cnt, day_od])
                else :
                    if value[9] == mcode_before :
                        values_per_day.append(value+[cnt, day_od])
                    else :
                        cnt += 1
                        values_per_day.append(value+[cnt, day_od])
                        mcode_before = value[9]
            df_+= values_per_day
            day_od += 1
        df_ = pd.DataFrame(df_, columns=df.columns.tolist()+['pgm_order', 'day_order'])

    else :
        date_unique = df.datetime.dt.date.unique().tolist()
        
        a = [100037, 100036, 100038]; b = [100039, 100038, 100037, 100036]
        c = [100037, 100039]; d = [100662, 100663]
        trouble_no = a+b+c+d
        
        df_ = []; day_od = 1
        for i in range(len(date_unique)-1) :
        
            date = date_unique[i]
            date_1 = date_unique[i+1]
        
            temp = df[df.datetime >= datetime.datetime(
                date.year, date.month, date.day, 6, 0)]
            temp = temp[temp.datetime <= datetime.datetime(
                date_1.year, date_1.month, date_1.day, 2, 20)]
        
            # mcode가 같으면 같은 숫자
            cnt=1; mcode_before=0; values_per_day=[]
            for i, value in enumerate(temp.values.tolist(), 1) :
                if mcode_before == 0 :
                    values_per_day.append(value+[cnt, day_od])
                    mcode_before = value[9]
                elif value[9] in trouble_no:
                    one_before = temp.values.tolist()[i-2][9]
                    if (one_before not in trouble_no) \
                        or ((value[9] in a) and (one_before not in a)) \
                            or ((value[9] in b) and (one_before not in b)) \
                                or ((value[9] in c) and (one_before not in c)) \
                                    or ((value[9] in d) and (one_before not in d)) :
                            cnt += 1
                    values_per_day.append(value+[cnt, day_od])
                else :
                    if value[9] == mcode_before :
                        values_per_day.append(value+[cnt, day_od])
                    else :
                        cnt += 1
                        values_per_day.append(value+[cnt, day_od])
                        mcode_before = value[9]
            df_+= values_per_day
            day_od += 1
        df_ = pd.DataFrame(df_, columns=df.columns.tolist()+['pgm_order', 'day_order'])

    repeate_info = df_.groupby(['datetime', 'pgm_order']).count().mcode.reset_index().values.tolist()
    repeat_order = []; pgm_before = 0
    for i in range(len(repeate_info)) :
        if repeate_info[i][1] != pgm_before :
            rep = 1
        else :
            rep += 1
        for _ in range(repeate_info[i][2]) :
            repeat_order.append(rep)
        pgm_before = repeate_info[i][1]
    df_['repeat_order'] = pd.Series(repeat_order)     

    return df_

#### (4) to vertorize product names

In [8]:
def get_pname_token_df(df_, pname_cols=None, pname_vocab=None, train=True) :
    
    df = df_.copy()
    
    if train :
        
        df['pname'] = df['pname'].str.replace(f'[{string.punctuation}]', ' ')
        df['pname'] = df['pname'].str.replace('  ', ' ')
        df['pname'] = df['pname'].str.strip()
        
        pname_unique = df.pname.unique().tolist()
        pname_spaced = {pn:'' for pn in pname_unique}
        for pn in pname_unique :
            pname_spaced[pn] = spacing(pn)
        
        df = df.replace({"pname": pname_spaced})
        
        df['pname'] = df['pname'].str.replace('삼성노트북', '삼성 노트북')
        df['pname'] = df['pname'].str.replace('쿠쿠전기밥솥', '쿠쿠 전기밥솥')
        df['pname'] = df['pname'].str.replace('한 샘', '한샘')
        df['pname'] = df['pname'].str.replace('삼성5도어', '삼성 5도어')
        df['pname'] = df['pname'].str.replace('에 버홈', '에버홈')
        df['pname'] = df['pname'].str.replace('휴롬퀵스퀴저', '휴롬 퀵스퀴저')
        df['pname'] = df['pname'].str.replace('매직 쉐프', '매직쉐프')
        df['pname'] = df['pname'].str.replace('쿠진나이프케어', '쿠진 나이프케어')
        df['pname'] = df['pname'].str.replace('에 지리', '에지리')
        df['pname'] = df['pname'].str.replace('린 나이', '린나이')
        df['pname'] = df['pname'].str.replace('한삼인순홍삼진7박스', '한삼인 순홍삼진7박스')
        df['pname'] = df['pname'].str.replace('미니건조기', '미니 건조기')
        
        # => pname bigram
        vectorizer = CountVectorizer(min_df=30, ngram_range=(1, 2))
        features = vectorizer.fit_transform(df['pname'].values.tolist())
        
        pname_bigram_df = pd.DataFrame(features.toarray())
        pname_bigram_df.columns = ['pname_'+str(i) for i in pname_bigram_df.columns]
        
        pname_voab = vectorizer.vocabulary_
        return pname_bigram_df, pname_voab

    else : # testset 만들 때

        df['pname'] = df['pname'].str.replace(f'[{string.punctuation}]', ' ')
        df['pname'] = df['pname'].str.replace('  ', ' ')
        df['pname'] = df['pname'].str.strip()
        
        pname_unique = df.pname.unique().tolist()
        pname_spaced = {pn:'' for pn in pname_unique}
        for pn in pname_unique :
            pname_spaced[pn] = spacing(pn)
        
        df = df.replace({"pname": pname_spaced})
        
        df['pname'] = df['pname'].str.replace('삼성노트북', '삼성 노트북')
        df['pname'] = df['pname'].str.replace('쿠쿠전기밥솥', '쿠쿠 전기밥솥')
        df['pname'] = df['pname'].str.replace('한 샘', '한샘')
        df['pname'] = df['pname'].str.replace('삼성5도어', '삼성 5도어')
        df['pname'] = df['pname'].str.replace('에 버홈', '에버홈')
        df['pname'] = df['pname'].str.replace('휴롬퀵스퀴저', '휴롬 퀵스퀴저')
        df['pname'] = df['pname'].str.replace('매직 쉐프', '매직쉐프')
        df['pname'] = df['pname'].str.replace('쿠진나이프케어', '쿠진 나이프케어')
        df['pname'] = df['pname'].str.replace('에 지리', '에지리')
        df['pname'] = df['pname'].str.replace('린 나이', '린나이')
        df['pname'] = df['pname'].str.replace('한삼인순홍삼진7박스', '한삼인 순홍삼진7박스')
        df['pname'] = df['pname'].str.replace('미니건조기', '미니 건조기')
        
        # => pname bigram
        vectorizer = CountVectorizer(ngram_range=(1, 2))
        features = vectorizer.fit_transform(df['pname'].values.tolist())
        testset_pname_vocab = vectorizer.vocabulary_
        
        pname_idx = []
        testset_idx_to_train_idx = dict()
        for k, v in testset_pname_vocab.items() :
            if k in pname_vocab :
                pname_idx.append(testset_pname_vocab[k])
                testset_idx_to_train_idx[testset_pname_vocab[k]] = pname_vocab[k]
        
        temp = pd.DataFrame(features.toarray())
        temp = temp.iloc[:,pname_idx]
        temp.rename(columns=testset_idx_to_train_idx, inplace=True)
        temp.columns = ['pname_'+str(i) for i in temp.columns]
        
        pname_bigram_df = pd.DataFrame(columns=pname_cols)
        pname_bigram_df_cols = temp.columns.tolist()
        for col in pname_bigram_df.columns :
            if col in pname_bigram_df_cols : 
                pname_bigram_df[col] = temp[col]
            else :
                pname_bigram_df[col] = 0
        
        return pname_bigram_df

#### (5) to create pseudo mcode/pcode of testset

In [9]:
def make_pseudo_m_pcode(df_, pname_bigram_df, test_month_dorders=None, pseudo_mcode_model=None, pseudo_pcode_model=None, 
                        m_cols=None, p_cols=None, train=True, train_validation=False) :

    if train and (not train_validation) : # testset(2020.6)의 유사코드를 부여하기 위해 모델을 생성할 때
        
        df = df_.copy()
        df['brand'] = df['brand'].astype('str')
        
        train_mcode_df = pd.concat([pd.get_dummies(df['brand']), 
                                    df[['mcode', 'price']], 
                                    pname_bigram_df],
                                   axis=1)

        train_pcode_df = pd.concat([pd.get_dummies(df['brand']), 
                                    df[['pcode', 'price']], 
                                    pname_bigram_df],
                                   axis=1)
        
        train_mcodes = train_mcode_df.mcode; del train_mcode_df['mcode']
        pseudo_mcode_model = KNeighborsClassifier(n_neighbors=3).fit(train_mcode_df.values, train_mcodes.values)

        train_pcodes = train_pcode_df.pcode; del train_pcode_df['pcode']
        pseudo_pcode_model = KNeighborsClassifier(n_neighbors=3).fit(train_pcode_df.values, train_pcodes.values) 
        
        m_cols = train_mcode_df.columns.tolist()
        p_cols = train_pcode_df.columns.tolist()
        
        return pseudo_mcode_model, pseudo_pcode_model, m_cols, p_cols, train_mcodes.unique().tolist(), train_pcodes.unique().tolist()

    
    elif not train : # testset(2020.6)에 유사코드 부여할 때

        df = df_.copy()
        df['brand'] = df['brand'].astype('str')

        mp_code_raw = df[['mcode', 'pcode']]        
        test_mcode_df = pd.concat([pd.get_dummies(df['brand']), 
                                   df[['mcode', 'price']], 
                                   pname_bigram_df],
                                  axis=1).drop_duplicates().reset_index(drop=True)

        test_pcode_df = pd.concat([pd.get_dummies(df['brand']), 
                                   df[['pcode', 'price']], 
                                   pname_bigram_df],
                                  axis=1).drop_duplicates().reset_index(drop=True)

        real_mcodes = test_mcode_df.mcode.values.tolist()
        real_pcodes = test_pcode_df.pcode.values.tolist()
        
        print('testset 구성 중..')
        test_mcols = test_mcode_df.columns.tolist()
        test_pcols = test_pcode_df.columns.tolist()
        for c in m_cols :
            if c not in test_mcols :
                test_mcode_df[c] = pd.Series()
                test_mcode_df[c] = 0
        for c in p_cols :
            if c not in test_pcols :
                test_pcode_df[c] = pd.Series()
                test_pcode_df[c] = 0

        test_mcode_df = test_mcode_df[m_cols]
        test_pcode_df = test_pcode_df[p_cols]

        print('유사코드 생성 중..')
        pseudo_mcodes = pseudo_mcode_model.predict(test_mcode_df.values)
        pseudo_pcodes = pseudo_pcode_model.predict(test_pcode_df.values)
        pseudo_mcode_df = pd.DataFrame({'real':real_mcodes, 'pseudo':pseudo_mcodes})
        pseudo_pcode_df = pd.DataFrame({'real':real_pcodes, 'pseudo':pseudo_pcodes})
                
        return df, mp_code_raw, pseudo_mcode_df, pseudo_pcode_df

#### (6) to optimize broadcast programming

In [10]:
def make_input_for_optimal(trainset, mp_code_raw, month_test, pseudo_pcode_dict, pseudo_mcode_dict,
                           tr_cols, search_cols, token_cols, trend_cols, pre_cols,
                           day_orders_for_optimize, full_day_orders_for_month, day=True) :

    train_subset = trainset.copy()
    
    # -- 실제 m/pcode로 치환
    train_subset['pcode'] = mp_code_raw['pcode']
    train_subset['mcode'] = mp_code_raw['mcode']
    
    # day_order(1~30), pgm_order(1~20), repeat_order(1~3)
    time_idx = [[d,p,r] 
                for d in range(full_day_orders_for_month[0], full_day_orders_for_month[-1]+1) 
                for p in range(1,21) 
                for r in range(1,4)]
    time_idx = pd.DataFrame(time_idx, columns=['day_order', 'pgm_order', 'repeat_order'])
    
    get_product_groups = train_subset.groupby(['day_order', 'pgm_order']).nunique().pcode.reset_index()
    get_product_groups.columns = ['day_order', 'pgm_order', 'pcode_cnt']
    
    pgroups_for_day_orders = [] # 상품 그룹
    for row in get_product_groups.values :
        if day : # 하루 편성 이라면
            if row[0] == day_orders_for_optimize :
                pgroups_for_day_orders.append(train_subset.loc[(train_subset.day_order == row[0]) 
                                                               & (train_subset.pgm_order == row[1])
                                                                & (trainset.sales_0 == 0),:].pcode.unique().tolist())
        else : # 주 단위 편성이라면
            if day_orders_for_optimize[0] <= row[0] <= day_orders_for_optimize[-1] :
                pgroups_for_day_orders.append(train_subset.loc[(train_subset.day_order == row[0]) 
                                                               & (train_subset.pgm_order == row[1])
                                                                & (trainset.sales_0 == 0),:].pcode.unique().tolist())
    
    product_groups_unique = list(list(map(lambda i: list(sorted(i)), pgroups_for_day_orders)))    
    
    if day : # 하루 편성이라면
        time_idx_sub = time_idx.loc[(time_idx.day_order == day_orders_for_optimize), :]
    else : # 일주일 편성이라면
        time_idx_sub = time_idx.loc[(time_idx.day_order >= day_orders_for_optimize[0]) 
                                    & (time_idx.day_order <= day_orders_for_optimize[-1]), :]
    
    rows = []
    for group in product_groups_unique :
        for row in time_idx_sub.values.tolist() :
            for i in range(len(group)) :
                rows.append(row + [group[i]])
    
    testset_frame = pd.DataFrame(rows, columns=['day_order', 'pgm_order', 'repeat_order', 'pcode'])
    testset_frame = testset_frame.sort_values(by=['day_order', 'pgm_order', 'pcode']).reset_index(drop=True)
    
    # 중복 제거
    testset_frame = testset_frame.drop_duplicates().reset_index(drop=True)
    
    # -- day_order, pgm_order에 대응되는 시간변수
    time_vars = ['month', 'weekday', 'week_order', 'holiday'] + search_cols + trend_cols
    time_vars_df = train_subset[['day_order', 'pgm_order']+time_vars].drop_duplicates().reset_index(drop=True)
    time_vars_df['pgm_order'] = time_vars_df['pgm_order'].astype('int')
    testset_frame['pgm_order'] = testset_frame['pgm_order'].astype('int')
    testset_frame = testset_frame.merge(time_vars_df, on=['day_order', 'pgm_order'], how='left')

    externals = ['ta', 'rn_or_not', 'ws', 'humid', 'dsnw', 'munzi', 'watching_rate']
    train_subset['pgm_order'] = train_subset['pgm_order'].astype('int')
    ex_vars_df = train_subset.groupby(['day_order', 'pgm_order']).mean()[externals].reset_index()
    testset_frame = testset_frame.merge(ex_vars_df, on=['day_order', 'pgm_order'], how='left')
    
    # -- pcode에 대응되는 상품변수
    product_vars = ['mcode', 'category', 'price', 'high_price', 'low_price', 'price_bin', 
                    'first_sale', 'sales_0_before', 'gender', 'payment_opt', 'vol_under5', 'vol_under10', 'sales_under100'] + token_cols + pre_cols
    if day :
        product_vars_df = train_subset.loc[(train_subset.day_order == day_orders_for_optimize) 
                                           & (trainset.sales_0 == 0), :][['pcode']+product_vars].drop_duplicates().reset_index(drop=True)
    else :
        product_vars_df = train_subset.loc[(train_subset.day_order >= day_orders_for_optimize[0]) 
                                           & (train_subset.day_order <= day_orders_for_optimize[-1]) 
                                           & (trainset.sales_0 == 0), :][['pcode']+product_vars].drop_duplicates().reset_index(drop=True)

    product_vars_df = product_vars_df.groupby('pcode').last().reset_index()
    product_vars_df['pcode'] = product_vars_df['pcode'].astype('int')
    testset_frame['pcode'] = testset_frame['pcode'].astype('int')
    testset_frame = testset_frame.merge(product_vars_df, on=['pcode'], how='left')  

    # duration=20
    testset_frame['duration'] = 20
    
    # p_cnt
    p_cnt_dict = {}
    for group in product_groups_unique :
        for item in group :
            p_cnt_dict[item] = len(group)
    testset_frame = testset_frame.merge(pd.DataFrame({'pcode':list(p_cnt_dict.keys()), 
                                                      'p_cnt':list(p_cnt_dict.values())}), on='pcode')
    
    # accumed_cnt
    acc_cnt = {}
    if day : # 하루 편성이라면
        temp = train_subset.loc[(train_subset.day_order == day_orders_for_optimize) 
                                & (trainset.sales_0 == 0),:][['pcode', 'accumed_cnt']]
    else : # 일주일 편성이라면
        temp = train_subset.loc[(train_subset.day_order >= day_orders_for_optimize[0]) 
                                & (train_subset.day_order <= day_orders_for_optimize[-1]) 
                                & (trainset.sales_0 == 0), :][['pcode', 'accumed_cnt']]

    for p in list(p_cnt_dict.keys()) :
        v = temp[temp.pcode == str(p)].accumed_cnt.values.tolist()
        if v :
            acc_cnt[p] = v[-1]
        else :
            acc_cnt[p] = 1
    testset_frame = testset_frame.merge(pd.DataFrame({'pcode':list(acc_cnt.keys()), 'accumed_cnt':list(acc_cnt.values())}), on='pcode')
    frame = testset_frame.copy()
    
    group_idx = {i:v for i, v in enumerate(product_groups_unique, 1)}
    
    testset_frame['pcode'] = testset_frame['pcode'].replace(pseudo_pcode_dict)
    testset_frame['mcode'] = testset_frame['mcode'].replace(pseudo_mcode_dict)

    
    # 사용할 변수 정의
    dummy_cols = ['month', 'weekday', 'duration', 'category', 'gender', 'payment_opt',
                  'pgm_order', 'repeat_order', 'week_order', 'price_bin', 'high_price', 'low_price', 'mcode', 'pcode']
    numeric_cols = ['p_cnt', 'price', 'accumed_cnt']
    binary_cols = ['holiday', 'first_sale', 'sales_0_before', 'vol_under5', 'vol_under10', 'sales_under100']
    pre_cols = ['pre_volume_median', 'pre_volume_max', 'pre_volume_min', 'pre_volume_var',
                'pre_sales_median', 'pre_sales_max', 'pre_sales_min', 'pre_sales_var']
    weather_cols = ['ta', 'rn_or_not', 'ws', 'humid', 'dsnw', 'munzi']
    w_rate = ['watching_rate']
    
    testset_frame['week_order'] = testset_frame['week_order'].astype('int')
    testset_frame['month'] = testset_frame['month'].astype('int') 
    testset_frame['duration'] = testset_frame['duration'].astype('float') 
    
    # 변수 데이터셋 생성
    df = pd.DataFrame()
    for col in dummy_cols :
        df[col] = pd.Series()
        df[col] = testset_frame[col].astype('str')
        df[col] = df[col].astype('str')
    df_dummies = pd.get_dummies(df[dummy_cols])
    df_numeric = testset_frame[numeric_cols + weather_cols + w_rate + token_cols + pre_cols + search_cols + trend_cols]

    features = pd.concat([df_dummies,
                          testset_frame[binary_cols], 
                          df_numeric
                         ],
                         axis=1)    

    features_ = pd.DataFrame(columns=tr_cols)
    test_cols = features.columns.tolist()
    for c in tr_cols :
        if c in test_cols :
            features_[c] = features[c]
        else :
            features_[c] = pd.Series()
            features_[c] = 0

    cols = ['month_'+str(i) for i in range(1,13) if i != month_test]
    for c in cols :
        features_[c] = features_[c].fillna(0).astype('int')

    return features_, frame, group_idx, pgroups_for_day_orders

### - preprocessing(1)

In [11]:
trainset = preprocessing(trainset, train=True)
trainset = date_split(trainset, train=True)
trainset = day_pgm_repeat_order(trainset, train=True)

### - preprocessing(2) + make variables

In [12]:
# 무형상품 제거
trainset = trainset[trainset.category != '무형'].reset_index(drop=True)

# EDA를 위한 판매량 칼럼 추가
trainset['sales_volume'] = trainset.sales / trainset.price

### - sales_0 pcode list (취급액이 0인 적이 있는 상품 리스트)

In [13]:
sales_0_df = trainset[trainset.sales_0 == 1][['pcode', 'pname']].drop_duplicates().reset_index(drop=True)
sales_0_pcodes = sales_0_df['pcode'].values.tolist()

### VAR 1) week_order
주(week)차 정보

In [14]:
week_order = []
for i in trainset.day :
    if 1 <= i <= 7 :
        week_order.append(1)
    elif 8 <= i <= 15 :
        week_order.append(2)
    elif 16 <= i <= 23 :
        week_order.append(3)
    else :
        week_order.append(4)
trainset['week_order'] = pd.Series(week_order)

### VAR 2) holiday
휴일 여부 / 공휴일 포함

In [15]:
holiday = []
for i, j in zip(trainset.datetime.dt.date, trainset.weekday) :
    if j in ('sat', 'sun') :
        holiday.append(1)
    elif str(i) in ('2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', 
                    '2019-03-01', '2019-05-06', '2019-05-12', '2019-06-06', 
                    '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', 
                    '2019-10-03', '2019-10-09', '2019-12-25') :
        holiday.append(1)
    else :
        holiday.append(0)
trainset['holiday'] = pd.Series(holiday)

### VAR 3) high price / low price
동일 방송에서 여러개의 상품을 판매하고 그 상품의 가격들이 다를 때, 가장 비싼 상품인지 / 가장 저렴한 상품인지의 여부


In [16]:
price_nunique = trainset.groupby(['day_order', 'pgm_order']).nunique().price.reset_index()
price_nunique = price_nunique[price_nunique.price > 1].reset_index(drop=True)

use_nan = None; high_price = []; low_price = []
cols = trainset.columns.tolist()
for row in trainset.values :
    if price_nunique.loc[(price_nunique.day_order == row[cols.index('day_order')]) & (price_nunique.pgm_order == row[cols.index('pgm_order')]), :].shape[0] :
        temp = trainset.loc[(trainset.day_order == row[cols.index('day_order')]) & (trainset.pgm_order == row[cols.index('pgm_order')]), :]
        if row[cols.index('price')] == temp.price.max() :
            high_price.append(1)
        else :
            high_price.append(0)
        if row[cols.index('price')] == temp.price.min() :
            low_price.append(1)
        else :
            low_price.append(0)            
    else :
        high_price.append(2) # 해당 없음
        low_price.append(2) # 해당 없음

trainset['high_price'] = pd.Series(high_price)
trainset['low_price'] = pd.Series(low_price)

### VAR 4) price_bin (가격 구간화)

In [17]:
qcut_idx_df = pd.DataFrame({'qcut':pd.qcut(trainset.price, 10), 
                            'label':LabelEncoder().fit_transform(pd.qcut(trainset.price, 10))}).drop_duplicates().reset_index(drop=True)

In [18]:
trainset['price_bin'] = LabelEncoder().fit_transform(pd.qcut(trainset.price, 10))

### VAR 5) p_cnt (함께 판매된 상품의 개수)

In [19]:
temp_df = trainset.groupby('datetime').count().pcode.reset_index()
temp_df.columns = ['datetime', 'p_cnt']
trainset = trainset.merge(temp_df, on=['datetime'])

### VAR 6) accumed_cnt/fist_sale (누적 방송 횟수/첫 방송 여부)

In [20]:
pcode_per_day = trainset.groupby(['day_order', 'pgm_order', 'pcode']).count().reset_index()[['day_order', 'pgm_order', 'pcode']]

cum_sale = []; first_sale=[]; no = 1
for i in pcode_per_day.values :
    cnt = 1
    temp = pcode_per_day.loc[(pcode_per_day.day_order < i[0]) & (pcode_per_day.pcode == i[2]), :].values
    check = temp.tolist()
    if check :
        first_sale.append(0)
        cum_sale.append(len(temp))
    else :
        first_sale.append(1)
        cum_sale.append(0)

pcode_per_day['accumed_cnt'] = pd.Series(cum_sale)
pcode_per_day['first_sale'] = pd.Series(first_sale)

trainset = trainset.merge(pcode_per_day, on=['day_order', 'pgm_order', 'pcode'], how='left')

### VAR 7) 이전 판매량 관련 변수

In [21]:
vol_pre_median = []; vol_pre_min = []; vol_pre_max = []; vol_pre_var = []
for d, p in trainset[['day_order', 'pcode']].values :
    temp1 = trainset.loc[(trainset.pcode == p) & (trainset.day_order < d),:].sales_volume.values
    check = temp1.tolist()
    if check :
        vol_pre_median.append(np.median(temp1))
        vol_pre_max.append(temp1.max())
        vol_pre_min.append(temp1.min())
        vol_pre_var.append(temp1.var())
    else :
        vol_pre_median.append(None)
        vol_pre_max.append(None)
        vol_pre_min.append(None)
        vol_pre_var.append(None)

trainset['pre_volume_median'] = vol_pre_median
trainset['pre_volume_max'] = vol_pre_max
trainset['pre_volume_min'] = vol_pre_min
trainset['pre_volume_var'] = vol_pre_var

### VAR 8) 이전 취급액 관련 변수

In [22]:
sales_pre_median = []; sales_pre_min = []; sales_pre_max = []; sales_pre_var = []
for d, p in trainset[['day_order', 'pcode']].values :
    temp1 = trainset.loc[(trainset.pcode == p) & (trainset.day_order < d),:].sales.values
    check = temp1.tolist()
    if check :
        sales_pre_median.append(np.median(temp1))
        sales_pre_max.append(temp1.max())
        sales_pre_min.append(temp1.min())
        sales_pre_var.append(temp1.var())
    else :
        sales_pre_median.append(None)
        sales_pre_max.append(None)
        sales_pre_min.append(None)
        sales_pre_var.append(None)

trainset['pre_sales_median'] = sales_pre_median
trainset['pre_sales_max'] = sales_pre_max
trainset['pre_sales_min'] = sales_pre_min
trainset['pre_sales_var'] = sales_pre_var

### VAR 9) 상품 당 duration

In [23]:
trainset['dur_per_p'] = trainset.duration / trainset.p_cnt

### VAR 10) 세일 폭

In [24]:
s = trainset.groupby('pname').price.unique()
s = s[s.apply(lambda x : len(x) > 1)]

def myfunc(df) :
    max_sales = df.price.max()
    promotion_ratio = (max_sales - df.price)/max_sales
    price = df.price
    
    return pd.DataFrame({
        'pname' : df.pname,
        'price' : price,
        'promotion_ratio' : promotion_ratio
    })

상품가격할인율 = trainset.groupby(['pname']).apply(myfunc)
ratio_dict = 상품가격할인율.drop_duplicates(['pname', 'price']).set_index(['pname', 'price'])

tmp = []
for i,r in trainset.iterrows() :
    tmp.append(ratio_dict.loc[(r['pname'], r['price'])][0])

trainset['promotion_ratio'] = tmp

### VAR 11) 외부변수 생성

In [25]:
weather.rename(columns={"hm": "humid"}, inplace=True)

weather = weather.rename(columns={'tm':'datetime'})
munzi.columns = ['stdid', 'stdnm', 'datetime', 'munzi']
wehter_g = weather.groupby('datetime').mean().reset_index()[['datetime', 'ta', 'rn', 'ws', 'humid', 'ss', 'icsr', 'dsnw']]
munzi_g = munzi.groupby('datetime').mean().reset_index()[['datetime', 'munzi']]

wehter_g['datetime'] = pd.to_datetime(wehter_g.datetime)
munzi_g['datetime'] = pd.to_datetime(munzi_g.datetime)
train_dt = pd.DataFrame({'datetime':trainset.datetime.dt.strftime('%Y-%m-%d %H:00:00')})
train_dt['datetime'] = pd.to_datetime(train_dt.datetime)

weather_features = train_dt.merge(wehter_g, on='datetime', how='left')
munzi_features = train_dt.merge(munzi_g, on='datetime', how='left')
weather_features['munzi'] = munzi_features['munzi']

del weather_features['ss']; del weather_features['icsr']
weather_features['rn'] = [1 if i > 0 else 0 for i in weather_features['rn']]
weather_features['dsnw'] = [1 if i > 0 else 0 for i in weather_features['dsnw']]
trainset = pd.concat([trainset, weather_features.iloc[:,1:]], axis=1)


# 시청률 변수 생성
watching_rate = watching_rate_data.set_index(watching_rate_data['시간대']).iloc[:-1,:-1]
watching_rate = watching_rate.iloc[:,1:].unstack().reset_index()
watching_rate.columns = ['date', 'time', 'w_rate']
watching_rate['datetime'] = watching_rate.date+' '+watching_rate.time
del watching_rate['date']; del watching_rate['time']
watching_rate = watching_rate[['datetime', 'w_rate']]

watching_rate['datetime'] = pd.to_datetime(watching_rate['datetime'])


time_index = pd.DataFrame({'start':pd.to_datetime(pd.Series(trainset.datetime.unique().astype('str').tolist())), 
                           'end':pd.to_datetime(pd.Series(trainset.datetime.unique()[1:].astype('str').tolist() + ['2020-01-01T00:20:00.000000000']))})

wr_sum = [] # 한 duration 내 총 시청률 합계
for row in time_index.values :
    wr_sum.append(watching_rate.loc[(watching_rate.datetime >= row[0]) & (watching_rate.datetime < row[1]),:].w_rate.sum())
time_index['watching_rate'] = pd.Series(wr_sum)
del time_index['end']
time_index.columns = ['datetime', 'watching_rate']

# -- 시청률 변수 추가
trainset = trainset.merge(time_index, on='datetime', how='left')

### VAR 12) 상품명 토큰변수

In [26]:
token_dict = {'token0': '10인용', 'token1': '6인용', 'token2': '4인용', 'token3': '3인용', 
              'token4': '6형', 'token5': '7형', 
              'token6': '75UK', 'token7': '70UK', 'token8': '65UK', 'token9': '55UK', 'token10': 'FQ17', 'token11': 'FQ19',
              'token12': '스탠드형', 'token13': '2IN1형',
              'token14': 'S10JK', 'token15': 'S06JK',
              'token16': '킹', 'token17': '퀸', 'token18': '슈퍼싱글',
              'token19': 'EV 040', 'token20': 'EV 020',
              'token21': '16R5773WSR', 'token22': '16R5773WSK',
              'token23': '홈멀티', 'token24': '스탠드',
              'token25': '아동',
              'token26': '2구', 'token27': '3구'}

for k, v in token_dict.items() :
    trainset[k] = pd.Series()
    temp = []
    for p in trainset.pname :
        if v in p :
            temp.append(1)
        else :
            temp.append(0)
    trainset[k] = temp

### VAR 13) SALES_0 에 등장한 적 있는지의 여부

In [27]:
sales_0_before = []

for i in trainset.pcode :
    if i in sales_0_df['pcode'].values :
        sales_0_before.append(1)
    else :
        sales_0_before.append(0)

trainset['sales_0_before'] = sales_0_before

### VAR 14) 남성상품/여성상품/해당없음

In [28]:
pat = re.compile('(남성)|(여성)')
pat_male = re.compile('남성')
pat_female = re.compile('여성')

성별_리스트 = []
for 상품명 in trainset.pname :
    if '남성' in 상품명 : 
        성별_리스트.append(1)
    elif '여성' in 상품명 : 
        성별_리스트.append(2)
    else :
        성별_리스트.append(0)

trainset['gender'] = 성별_리스트

### VAR 15) 결제 방식

In [29]:
무이자일시불 = []
무이자표현 = ['무이자', '무)']
일시불표현 = ['일시불', '일)']
for i in trainset.pname :
    초기값 = 0
    for j in 무이자표현 :
        if j in i :
            초기값 = 1 # 무이자
    for j in 일시불표현 :
        if j in i :
            초기값 = 2 # 일시불
    무이자일시불.append(초기값)

trainset['payment_opt'] = 무이자일시불

### VAR 17) search(상품 검색량), trend(사회적 계절성) 변수  추가

#### - trend

In [30]:
trend_df.rename(columns={'날짜':'datetime'}, inplace=True)
trend_dict.columns = ['key', 'trend']
trend_df['datetime'] = pd.to_datetime(trend_df['datetime'])

In [31]:
trainset = pd.concat([trainset,
                      pd.DataFrame({'datetime':pd.to_datetime(trainset.datetime.dt.date)}).merge(trend_df, on='datetime', how='left').iloc[:,1:]], 
                     axis=1)

#### - search

In [32]:
search_df.rename(columns={'날짜':'datetime'}, inplace=True)
search_dict.columns = ['key', 'trend']
search_df['datetime'] = pd.to_datetime(search_df['datetime'])

In [33]:
trainset = pd.concat([trainset,
                      pd.DataFrame({'datetime':pd.to_datetime(trainset.datetime.dt.date)}).merge(search_df, on='datetime', how='left').iloc[:,1:]], 
                     axis=1)

### VAR 18) 이전 판매기록의 변수화
- 판매량이 5보다 작았던 적이 있는 pcode이다 / 판매량이 10 보다 작았던 적이 있는 pcode 이다
- 취급액이 100만원보다 작았던 적이 있는 pcode이다

In [34]:
vol_under5_pcode = trainset[trainset.sales_volume < 6].pcode.unique().tolist()
vol_under10_pcode = trainset[trainset.sales_volume < 11].pcode.unique().tolist()
sales_under100_pcode = trainset[trainset.sales < 1000000].pcode.unique().tolist()

In [35]:
trainset['vol_under5'] = pd.Series([1 if i in vol_under5_pcode else 0 for i in trainset.pcode])
trainset['vol_under10'] = pd.Series([1 if i in vol_under10_pcode else 0 for i in trainset.pcode])
trainset['sales_under100'] = pd.Series([1 if i in sales_under100_pcode else 0 for i in trainset.pcode])

### VAR 19) 상품별 재방송(repeat_order)에 따른 증가율

In [36]:
up_ratio = []
product_pgm_index = trainset.groupby(['pcode', 'day_order', 'pgm_order']).count().reset_index()[['pcode', 'day_order', 'pgm_order']]
for c, d, p in product_pgm_index.values :
    temp = trainset.loc[(trainset.pcode == c) & (trainset.day_order == d) & (trainset.pgm_order == p) , :]
    up_ratio.append(temp.sales.tolist()[-1] / temp.sales.tolist()[0])

In [37]:
product_pgm_index['up_ratio'] = up_ratio

In [38]:
repeat_ratio_median = []; repeat_ratio_max = []; repeat_ratio_min = []; repeat_ratio_var = []
for values in trainset[['pcode', 'day_order' ,'pgm_order']].values :
    temp = product_pgm_index.loc[(product_pgm_index.pcode == values[0]) & (product_pgm_index.day_order <= values[0]), :]
    repeat_ratio_median.append(np.median(temp.up_ratio)); repeat_ratio_max.append(max(temp.up_ratio))
    repeat_ratio_min.append(min(temp.up_ratio)); repeat_ratio_var.append(np.var(temp.up_ratio))

In [39]:
trainset['repeat_ratio_median'] = repeat_ratio_median
trainset['repeat_ratio_max'] = repeat_ratio_max
trainset['repeat_ratio_min'] = repeat_ratio_min
trainset['repeat_ratio_var'] = repeat_ratio_var

### VAR 20) 브랜드

In [40]:
brand_df.columns = ['pname', 'brand', '']
brand_df = brand_df.iloc[:,:-1]

In [41]:
trainset = trainset.merge(brand_df, on='pname', how='left')

In [42]:
brand_dict = {b:l for b, l in zip(trainset.brand, LabelEncoder().fit_transform(trainset.brand))}
trainset['brand'] = LabelEncoder().fit_transform(trainset.brand)

###  preprocessing(3)

In [43]:
# 카테고리 => 한글
cate_dict = {'의류':'clothes', 
             '속옷':'under_wear', 
             '주방':'kitchen', 
             '농수축':'argo_farming', 
             '이미용':'beauty', 
             '가전':'electronics', 
             '생활용품':'living', 
             '건강기능':'health', 
             '잡화':'general_merchandise', 
             '가구':'furnitures', 
             '침구':'beds'}
trainset = trainset.replace({"category": cate_dict})

In [44]:
# duration 구간화
trainset['duration'] = trainset['duration'].round(-1)
trainset['duration'].replace({0:20, 40:30}, inplace=True)

## - Modeling
#### - 유사코드 부여를 위한 trainset 상품명 vector화 / 유사코드 모델 생성

In [45]:
pname_bigram_train_df, pname_voab = get_pname_token_df(trainset, train=True)

In [46]:
pseudo_mcode_model, pseudo_pcode_model, train_mcode_cols, train_pcode_cols, \
train_mcodes_uniques, train_pcodes_uniques = make_pseudo_m_pcode(trainset, pname_bigram_train_df,
                                                                 train=True, 
                                                                 train_validation=False)

### - Make Testset

In [47]:
testset = preprocessing(testset, train=False)
testset = date_split(testset, train=False)
testset = day_pgm_repeat_order(testset, train=False)

In [48]:
week_order = []
for i in testset.day :
    if 1 <= i <= 7 :
        week_order.append(1)
    elif 8 <= i <= 15 :
        week_order.append(2)
    elif 16 <= i <= 23 :
        week_order.append(3)
    else :
        week_order.append(4)
testset['week_order'] = pd.Series(week_order)

In [49]:
holiday = []
for i, j in zip(testset.datetime.dt.date, testset.weekday) :
    if j in ('sat', 'sun') :
        holiday.append(1)
    else :
        holiday.append(0)
testset['holiday'] = holiday

In [50]:
price_nunique = testset.groupby(['day_order', 'pgm_order']).nunique().price.reset_index()
price_nunique = price_nunique[price_nunique.price > 1].reset_index(drop=True)

use_nan = None; high_price = []; low_price = []
cols = testset.columns.tolist()
for row in testset.values :
    if price_nunique.loc[(price_nunique.day_order == row[cols.index('day_order')]) 
                         & (price_nunique.pgm_order == row[cols.index('pgm_order')]), :].shape[0] :
        temp = testset.loc[(testset.day_order == row[cols.index('day_order')]) 
                           & (testset.pgm_order == row[cols.index('pgm_order')]), :]
        if row[cols.index('price')] == temp.price.max() :
            high_price.append(1)
        else :
            high_price.append(0)
        if row[cols.index('price')] == temp.price.min() :
            low_price.append(1)
        else :
            low_price.append(0)            
    else :
        high_price.append(2) # 해당 없음
        low_price.append(2) # 해당 없음

testset['high_price'] = pd.Series(high_price)
testset['low_price'] = pd.Series(low_price)

In [51]:
price_bin = []
for i in testset.price :
    for q, l in qcut_idx_df.values :
        if i in q :
            price_bin.append(l)
            break
    else :
        if i <= q.left :
            price_bin.append(0)
        elif i > q.right :
            price_bin.append(9)
testset['price_bin'] = price_bin

In [52]:
temp_df = testset.groupby('datetime').count().pcode.reset_index()
temp_df.columns = ['datetime', 'p_cnt']
testset = testset.merge(temp_df, on=['datetime'], how='left')

In [53]:
testset = testset.merge(brand_df, on='pname', how='left')
testset['brand'] = testset['brand'].replace(brand_dict)

### - 유사 마더코드/상품코드 부여

In [54]:
pname_bigram_test_df = get_pname_token_df(testset, 
                                          pname_cols=pname_bigram_train_df.columns.tolist(), 
                                          pname_vocab=pname_voab, 
                                          train=False)

In [55]:
testset, mp_code_raw, pseudo_mcode_df, pseudo_pcode_df = make_pseudo_m_pcode(testset, 
                                                                             pname_bigram_test_df, 
                                                                             pseudo_mcode_model=pseudo_mcode_model, 
                                                                             pseudo_pcode_model=pseudo_pcode_model, 
                                                                             m_cols=train_mcode_cols, 
                                                                             p_cols=train_pcode_cols, 
                                                                             train=False, 
                                                                             train_validation=False)

testset 구성 중..
유사코드 생성 중..


In [56]:
# trainset에 없는 mcode, pcode의 경우, pseudo code 사전을 참조하여 대체

pseudo_mcode = []
for i in testset.mcode :
    if i in train_mcodes_uniques :
        pseudo_mcode.append(i)
    else :
        pseudo_mcode.append(pseudo_mcode_df[pseudo_mcode_df.real==i].pseudo.unique().tolist()[0])
        
pseudo_pcode = []
for i in testset.pcode :
    if i in train_pcodes_uniques :
        pseudo_pcode.append(i)
    else :
        pseudo_pcode.append(pseudo_pcode_df[pseudo_pcode_df.real==i].pseudo.unique().tolist()[0])

testset['mcode'] = pseudo_mcode
testset['pcode'] = pseudo_pcode

#### - accumed_cnt/fist_sale (누적 방송 횟수/첫 방송 여부)
유사코드 대체 후, 해당 코드의 방송 중 가장 마지막 방송의 값을 가져옴

In [57]:
accumed_cnt = []; first_sale = []
for p in testset.pcode :
    temp = trainset.loc[(trainset.pcode==p)]
    accumed_cnt.append(temp.accumed_cnt.values.tolist()[-1])
    first_sale.append(temp.first_sale.values.tolist()[-1])

In [58]:
testset['accumed_cnt'] = pd.Series(accumed_cnt)
testset['first_sale'] = pd.Series(first_sale)

#### - 이전 판매량 / 이전 취급액 / 이전 판매기록 관련 변수
유사코드 대체 후, 해당 코드의 방송 중 가장 마지막 방송의 값을 가져옴

In [59]:
# 이전 판매기록 관련 리스트
sales_0_before = []; vol_under5 = []; vol_under10 = []; sales_under100 = []
for i in testset.pcode :
    if i in sales_0_pcodes :
        sales_0_before.append(1)
    else :
        sales_0_before.append(0)
    if i in vol_under5_pcode :
        vol_under5.append(1)
    else :
        vol_under5.append(0)
    if i in vol_under10_pcode :
        vol_under10.append(1)
    else :
        vol_under10.append(0)
    if i in sales_under100_pcode :
        sales_under100.append(1)
    else :
        sales_under100.append(0)

repeat_cols = ['repeat_ratio_median', 'repeat_ratio_max', 'repeat_ratio_min', 'repeat_ratio_var']
pre_cols = ['pre_volume_median', 'pre_volume_max', 'pre_volume_min', 'pre_volume_var',
            'pre_sales_median', 'pre_sales_max', 'pre_sales_min', 'pre_sales_var']

repeat_ratio_median = []; repeat_ratio_max = []; repeat_ratio_min = []; repeat_ratio_var = []
pre_volume_median = []; pre_volume_max = []; pre_volume_min = []; pre_volume_var = []
pre_sales_median = []; pre_sales_max = []; pre_sales_min = []; pre_sales_var = []

for p in testset.pcode :
    temp = trainset.loc[(trainset.pcode==p)]
    repeat_ratio_median.append(temp.repeat_ratio_median.values.tolist()[-1])
    repeat_ratio_max.append(temp.repeat_ratio_max.values.tolist()[-1])
    repeat_ratio_min.append(temp.repeat_ratio_min.values.tolist()[-1])
    repeat_ratio_var.append(temp.repeat_ratio_var.values.tolist()[-1])
    pre_volume_median.append(temp.pre_volume_median.values.tolist()[-1])
    pre_volume_max.append(temp.pre_volume_max.values.tolist()[-1])
    pre_volume_min.append(temp.pre_volume_min.values.tolist()[-1])
    pre_volume_var.append(temp.pre_volume_var.values.tolist()[-1])
    pre_sales_median.append(temp.pre_sales_median.values.tolist()[-1])
    pre_sales_max.append(temp.pre_sales_max.values.tolist()[-1])
    pre_sales_min.append(temp.pre_sales_min.values.tolist()[-1])
    pre_sales_var.append(temp.pre_sales_var.values.tolist()[-1])

testset['sales_0_before'] = sales_0_before
testset['vol_under5'] = vol_under5
testset['vol_under10'] = vol_under10
testset['sales_under100'] = sales_under100
testset['repeat_ratio_median'] = repeat_ratio_median
testset['repeat_ratio_max'] = repeat_ratio_max
testset['repeat_ratio_min'] = repeat_ratio_min
testset['repeat_ratio_var'] = repeat_ratio_var
testset['pre_volume_median'] = pre_volume_median
testset['pre_volume_max'] = pre_volume_max
testset['pre_volume_min'] = pre_volume_min
testset['pre_volume_var'] = pre_volume_var
testset['pre_sales_median'] = pre_sales_median
testset['pre_sales_max'] = pre_sales_max
testset['pre_sales_min'] = pre_sales_min
testset['pre_sales_var'] = pre_sales_var

In [60]:
testset['dur_per_p'] = testset.duration / testset.p_cnt

In [61]:
s = testset.groupby('pname').price.unique()
s = s[s.apply(lambda x : len(x) > 1)]

def myfunc(df) :
    max_sales = df.price.max()
    promotion_ratio = (max_sales - df.price)/max_sales
    price = df.price
    
    return pd.DataFrame({
        'pname' : df.pname,
        'price' : price,
        'promotion_ratio' : promotion_ratio
    })

상품가격할인율 = testset.groupby(['pname']).apply(myfunc)
ratio_dict = 상품가격할인율.drop_duplicates(['pname', 'price']).set_index(['pname', 'price'])

tmp = []
for i, r in testset.iterrows() :
    tmp.append(ratio_dict.loc[(r['pname'], r['price'])][0])

testset['promotion_ratio'] = tmp
testset['promotion_ratio'].fillna(0, inplace=True)

In [62]:
munzi = pd.read_csv('./data/external_data/weather/munzi_pm10_utf.csv')
brand_df = pd.read_excel('./data/external_data/product_info/brand.xlsx').iloc[:,1:]

weather.rename(columns={"hm": "humid"}, inplace=True)

weather = weather.rename(columns={'tm':'datetime'})
munzi.columns = ['stdid', 'stdnm', 'datetime', 'munzi']
wehter_g = weather.groupby('datetime').mean().reset_index()[['datetime', 'ta', 'rn', 'ws', 'humid', 'ss', 'icsr', 'dsnw']]
munzi_g = munzi.groupby('datetime').mean().reset_index()[['datetime', 'munzi']]

wehter_g['datetime'] = pd.to_datetime(wehter_g.datetime)
munzi_g['datetime'] = pd.to_datetime(munzi_g.datetime)
test_dt = pd.DataFrame({'datetime':testset.datetime.dt.strftime('%Y-%m-%d %H:00:00')})
test_dt['datetime'] = pd.to_datetime(test_dt.datetime)

weather_features = test_dt.merge(wehter_g, on='datetime', how='left')
munzi_features = test_dt.merge(munzi_g, on='datetime', how='left')
weather_features['munzi'] = munzi_features['munzi']

del weather_features['ss']; del weather_features['icsr']
weather_features['rn'] = [1 if i > 0 else 0 for i in weather_features['rn']]
weather_features['dsnw'] = [1 if i > 0 else 0 for i in weather_features['dsnw']]
testset = pd.concat([testset, weather_features.iloc[:,1:]], axis=1)


# 시청률 변수 생성
watching_rate = watching_rate_data.set_index(watching_rate_data['시간대']).iloc[:-1,:-1]
watching_rate = watching_rate.iloc[:,1:].unstack().reset_index()
watching_rate.columns = ['date', 'time', 'w_rate']
watching_rate['datetime'] = watching_rate.date+' '+watching_rate.time
del watching_rate['date']; del watching_rate['time']
watching_rate = watching_rate[['datetime', 'w_rate']]

watching_rate['datetime'] = watching_rate['datetime'].str.replace('2019', '2020')    
watching_rate['datetime'] = pd.to_datetime(watching_rate['datetime'])
time_index = pd.DataFrame({'start':pd.to_datetime(pd.Series(testset.datetime.unique().astype('str').tolist())), 
                           'end':pd.to_datetime(pd.Series(testset.datetime.unique()[1:].astype('str').tolist() + ['2020-07-01T00:20:00.000000000']))})        

wr_sum = [] # 한 duration 내 총 시청률 합계
for row in time_index.values :
    wr_sum.append(watching_rate.loc[(watching_rate.datetime >= row[0]) & (watching_rate.datetime < row[1]),:].w_rate.sum())
time_index['watching_rate'] = pd.Series(wr_sum)
del time_index['end']
time_index.columns = ['datetime', 'watching_rate']

# -- 시청률 변수 추가
testset = testset.merge(time_index, on='datetime', how='left')

In [63]:
for k, v in token_dict.items() :
    testset[k] = pd.Series()
    temp = []
    for p in testset.pname :
        if v in p :
            temp.append(1)
        else :
            temp.append(0)
    testset[k] = temp

In [64]:
pat = re.compile('(남성)|(여성)')
pat_male = re.compile('남성')
pat_female = re.compile('여성')

성별_리스트 = []
for 상품명 in testset.pname :
    if '남성' in 상품명 : 
        성별_리스트.append(1)
    elif '여성' in 상품명 : 
        성별_리스트.append(2)
    else :
        성별_리스트.append(0)

testset['gender'] = 성별_리스트

In [65]:
무이자일시불 = []
무이자표현 = ['무이자', '무)']
일시불표현 = ['일시불', '일)']
for i in testset.pname :
    초기값 = 0
    for j in 무이자표현 :
        if j in i :
            초기값 = 1 # 무이자
    for j in 일시불표현 :
        if j in i :
            초기값 = 2 # 일시불
    무이자일시불.append(초기값)

testset['payment_opt'] = 무이자일시불

In [66]:
testset = pd.concat([testset,
                      pd.DataFrame({'datetime':pd.to_datetime(testset.datetime.dt.date)}).merge(trend_df, on='datetime', how='left').iloc[:,1:]], 
                     axis=1)

In [67]:
testset = pd.concat([testset,
                      pd.DataFrame({'datetime':pd.to_datetime(testset.datetime.dt.date)}).merge(search_df, on='datetime', how='left').iloc[:,1:]], 
                     axis=1)

In [68]:
# 카테고리 => 한글
cate_dict = {'의류':'clothes', 
             '속옷':'under_wear', 
             '주방':'kitchen', 
             '농수축':'argo_farming', 
             '이미용':'beauty', 
             '가전':'electronics', 
             '생활용품':'living', 
             '건강기능':'health', 
             '잡화':'general_merchandise', 
             '가구':'furnitures', 
             '침구':'beds', 
             '무형':'mu'}
testset = testset.replace({"category": cate_dict})

In [69]:
# duration 구간화
testset['duration'] = testset['duration'].round(-1)

### - make models

#### - make X features

In [70]:
dummy_cols = ['month', 'hour', 'minute', 'weekday', 'duration', 'mcode', 'pcode', 'category', 
              'pgm_order', 'repeat_order', 'week_order', 'holiday', 'high_price', 'low_price', 'price_bin']
numeric_binary_cols = ['p_cnt', 'accumed_cnt', 'first_sale', 'price',
                       'pre_volume_median', 'pre_volume_max', 'pre_volume_min', 'pre_volume_var', 
                       'pre_sales_median', 'pre_sales_max', 'pre_sales_min', 'pre_sales_var', 
                       'dur_per_p', 'promotion_ratio', 
                       'ta', 'rn', 'ws', 'humid', 'dsnw', 'munzi', 'watching_rate', 
                       'token0', 'token1', 'token2', 'token3', 'token4', 'token5', 'token6', 'token7', 'token8', 
                       'token9', 'token10', 'token11', 'token12', 'token13', 'token14', 'token15', 'token16', 
                       'token17', 'token18', 'token19', 'token20', 'token21', 'token22', 'token23', 'token24', 
                       'token25', 'token26', 'token27', 'sales_0_before', 'gender', 'payment_opt', 
                       'trend0', 'trend1', 'trend2', 'trend3', 'trend4', 'trend5', 'trend6', 'trend7', 'trend8', 
                       'search0', 'search1', 'search2', 'search3', 'search4', 'search5', 'search6', 'search7', 'search8', 
                       'search9', 'search10', 'search11', 'search12', 'search13', 'search14', 'search15', 'search16', 'search17', 
                       'search18', 'search19', 'search20', 'search21', 'search22', 'search23', 'search24', 'search25', 'search26', 
                       'search27', 'search28', 'search29', 'vol_under5', 'vol_under10', 'sales_under100', 
                       'repeat_ratio_median', 'repeat_ratio_max', 'repeat_ratio_min', 'repeat_ratio_var']

#### - trainset

In [71]:
dummy_df = pd.DataFrame()
for col in dummy_cols :
    dummy_df[col] = pd.Series()
    dummy_df[col] = trainset[col].astype('str')
dummy_df = pd.get_dummies(dummy_df)

In [72]:
features_train = pd.concat([dummy_df, trainset[numeric_binary_cols]], axis=1)
Y = trainset.sales

#### - testset

In [73]:
dummy_df_ = pd.DataFrame()
for col in dummy_cols :
    dummy_df_[col] = pd.Series()
    dummy_df_[col] = testset[col].astype('str')
dummy_df_ = pd.get_dummies(dummy_df_)

In [74]:
features_test_temp = pd.concat([dummy_df_, testset[numeric_binary_cols]], axis=1)

In [75]:
tr_cols = features_train.columns.tolist()
tt_cols = features_test_temp.columns.tolist()
features_test = pd.DataFrame(columns = tr_cols)

for c in tr_cols :
    if c in tt_cols :
        features_test[c] = features_test_temp[c]
    else :
        features_test[c] = pd.Series([0 for i in range(features_test_temp.shape[0])])

features_test = features_test[tr_cols]

### - get predicted values

In [76]:
hyper_params = {
    'task': 'train',
    'num_iterations':1500,
    'boosting_type': 'dart',
    'max_depth':11,
    'num_leaves':2023,
    'drop_rate' : 0.5,
    'objective': 'regression_l1',
    'tree_learner':'voting',
    'num_threads': -1,
    'metric': [mape],
    'feature_fraction':0.8,
    'learning_rate': 0.01
    }


train_data = Dataset(features_train.values, Y.values)
model = lightgbm.train(hyper_params, 
                       train_set=train_data,
                       verbose_eval=True)

pred = model.predict(features_test.values)
pred = np.array([features_test.price[i] if p < 0 else p for i, p in enumerate(pred)])

In [77]:
testset_to_submit = pd.read_excel('./data/02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', skiprows=1)

In [78]:
testset_to_submit['취급액'] = pred

In [79]:
testset_to_submit['취급액'] = pd.Series([None if p == '무형' else s for p, s in zip(testset_to_submit['상품군'], testset_to_submit['취급액'])])

In [80]:
testset_to_submit.to_csv('predict_result_.csv', index=False, encoding='utf-8')